In [81]:
import torch
LOAD_TRAIN_PATH2 = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\TrainVal\TRAIN2.pth"
LOAD_TRAIN_PATH = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\TrainVal\TRAIN3.pth"

LOAD_VAL_PATH = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\TrainVal\VAL3.pth"

train_data = torch.load(LOAD_TRAIN_PATH)
train_data2 = torch.load(LOAD_TRAIN_PATH2)
val_data = torch.load(LOAD_VAL_PATH)

load_train_patches = train_data['patches'] #(N_total, 3, 64, 64) #0~255
load_train_labels = train_data['masks'] #(N_total, 1, 64, 64) #0 or 1


load_train_patches2 = train_data2['patches'] #(N_total, 3, 64, 64) #0~255
load_train_labels2 = train_data2['masks'] #(N_total, 1, 64, 64) #0 or 1

load_val_patches = val_data['patches'] #(N_total, 3, 64, 64) #0~255
load_val_labels = val_data['masks'] #(N_total, 1, 64, 64) #0 or 1

load_train_patches = torch.cat([load_train_patches, load_train_patches2], dim=0)
load_train_labels = torch.cat([load_train_labels, load_train_labels2], dim=0)

C:\Users\ygz08\AppData\Local\Temp\ipykernel_3428\2910766302.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(LOAD_TRAIN_PATH)
C:\Users\ygz08\AppDa

In [82]:
if train_data and val_data :
    # 檢查訓練 Patches 維度
    # 確保是四個維度，且 Channel=3, H=64, W=64。N_total 不固定，用 -1 檢查
    assert load_train_patches.dim() == 4, \
        f"訓練 Patches 維度錯誤! 預期 4 維, 實際 {load_train_patches.dim()} 維."
    assert load_train_patches.shape[1:] == (3, 64, 64), \
        f"訓練 Patches Shape 錯誤! 預期 (N, 3, 64, 64), 實際 {load_train_patches.shape}."

    # 檢查訓練 Labels (Masks) 維度
    # 確保是四個維度，且 Channel=1, H=64, W=64
    assert load_train_labels.dim() == 4, \
        f"訓練 Labels 維度錯誤! 預期 4 維, 實際 {load_train_labels.dim()} 維."
    assert load_train_labels.shape[1:] == (1, 64, 64), \
        f"訓練 Labels Shape 錯誤! 預期 (N, 1, 64, 64), 實際 {load_train_labels.shape}."

    # 檢查訓練資料的批次大小是否一致
    assert load_train_patches.shape[0] == load_train_labels.shape[0], \
        f"訓練資料批次大小不一致! Patches: {load_train_patches.shape[0]}, Labels: {load_train_labels.shape[0]}."
        
    # 建議的資料型態檢查
    # Patches (輸入影像) 通常是 float
    # Labels (遮罩) 通常是 float 或 long (如果是類別索引)
    assert load_train_patches.dtype == torch.float32, \
        f"訓練 Patches 資料型態錯誤! 預期 torch.float32, 實際 {load_train_patches.dtype}."
    assert load_train_labels.dtype in [torch.float32, torch.long], \
        f"訓練 Labels 資料型態錯誤! 預期 torch.float32 或 torch.long, 實際 {load_train_labels.dtype}."

    # --- 🎯 檢查驗證資料 (VAL Data Assertions) ---

    # 檢查驗證 Patches 維度
    assert load_val_patches.dim() == 4, \
        f"驗證 Patches 維度錯誤! 預期 4 維, 實際 {load_val_patches.dim()} 維."
    assert load_val_patches.shape[1:] == (3, 64, 64), \
        f"驗證 Patches Shape 錯誤! 預期 (N, 3, 64, 64), 實際 {load_val_patches.shape}."

    # 檢查驗證 Labels (Masks) 維度
    assert load_val_labels.dim() == 4, \
        f"驗證 Labels 維度錯誤! 預期 4 維, 實際 {load_val_labels.dim()} 維."
    assert load_val_labels.shape[1:] == (1, 64, 64), \
        f"驗證 Labels Shape 錯誤! 預期 (N, 1, 64, 64), 實際 {load_val_labels.shape}."
        
    # 檢查驗證資料的批次大小是否一致
    assert load_val_patches.shape[0] == load_val_labels.shape[0], \
        f"驗證資料批次大小不一致! Patches: {load_val_patches.shape[0]}, Labels: {load_val_labels.shape[0]}."

    # 建議的資料型態檢查
    assert load_val_patches.dtype == torch.float32, \
        f"驗證 Patches 資料型態錯誤! 預期 torch.float32, 實際 {load_val_patches.dtype}."
    assert load_val_labels.dtype in [torch.float32, torch.long], \
        f"驗證 Labels 資料型態錯誤! 預期 torch.float32 或 torch.long, 實際 {load_val_labels.dtype}."

    print("✅ Train 有 {} 個樣本，Val 有 {} 個樣本。".format(load_train_patches.shape[0], load_val_patches.shape[0]))
    print("✅ 所有 Tensor 的維度與型態檢查通過！")

✅ Train 有 2421 個樣本，Val 有 330 個樣本。
✅ 所有 Tensor 的維度與型態檢查通過！


In [83]:
from torch.utils.data import TensorDataset, DataLoader

# 1. 確保數據類型和範圍正確
# patches (影像) 通常需要是 float 類型，labels (掩碼) 也是 float (因為 sigmoid 輸出)
# 並且將 0~255 的 patches 歸一化到 0~1 範圍 (如果尚未歸一化)

# 假設原始 patches 是 0~255 的 uint8 或 float 類型
# 這裡將其轉為 float 並歸一化 (如果您的預處理還沒做這步)
# 如果您的數據已經是歸一化的 float，可以跳過這一步的除以 255
load_train_patches = load_train_patches.float() / 255.0
load_val_patches = load_val_patches.float() / 255.0

load_train_labels = load_train_labels.float()
load_val_labels = load_val_labels.float()

# 2. 創建 TensorDataset
train_dataset = TensorDataset(load_train_patches, load_train_labels)
val_dataset = TensorDataset(load_val_patches, load_val_labels)

# 3. 創建 DataLoader
BATCH_SIZE = 32 # 可以根據你的 GPU 記憶體調整
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"訓練資料集大小: {len(train_dataset)}")
print(f"訓練批次數量: {len(train_loader)}")

訓練資料集大小: 2421
訓練批次數量: 76


In [84]:
import torch.nn.functional as F

# --- 1. 論文專用 Loss 函數 (Dice 形式) ---
# 遵守論文公式: Dice_Loss = 1 - (2 * V(Gt ∩ Seg) + η) / (V(Gt) + V(Seg) + η)
def paper_dice_loss(pred, target, eta=1e-6): # 使用 eta 替換 smooth，強調對應論文中的 η
    """
    計算論文中所示的 Dice 形式的損失函數 (Loss Function)。
    
    Args:
        pred (Tensor): 模型的預測機率 (0-1)。
        target (Tensor): 真實分割掩碼 (0/1)。
        eta (float): 論文中防止分母為零的非零參數 (平滑項)。
    """
    # 展平張量 (V(X) 相當於元素總和)
    pred_flat = pred.contiguous().view(-1)
    target_flat = target.contiguous().view(-1)
    
    # V(Gt ∩ Seg) = 交集
    intersection = (pred_flat * target_flat).sum()
    
    # V(Gt) + V(Seg) = 總和
    union = pred_flat.sum() + target_flat.sum()
    
    # 計算論文中的 Dice 公式 (注意這裡 pred 和 target 都是 soft probability/float)
    dice_numerator = 2. * intersection + eta
    dice_denominator = union + eta
    
    dice_score_term = dice_numerator / dice_denominator
    
    # 論文損失函數 = 1 - Dice Score Term
    loss = 1 - dice_score_term
    
    return loss

# --- 2. 移除 paper_dice_loss (不再使用 BCE) ---
# 在訓練迴圈中，直接使用 loss = paper_dice_loss(outputs, masks)

# --- 3. 評估指標 (Dice Score) 保留 ---
# 評估時仍然使用 Dice Score (通常是二值化後的結果)
def dice_score(pred, target, smooth=1e-6):
    """計算 Dice 係數 (Dice Coefficient)，用於性能評估 (通常使用二值化)。"""
    # 確保輸入是二元 (0 或 1)
    pred = (pred > 0.5).float()
    
    pred_flat = pred.contiguous().view(-1)
    target_flat = target.contiguous().view(-1)
    
    intersection = (pred_flat * target_flat).sum()
    dice = (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)
    
    return dice.item()

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 1. 基礎單元: DenseBlock (DB) ---
class DenseBlock(nn.Module):
    # growth_rate 就是論文中的 n (8, 16, 32, ...)
    def __init__(self, in_channels, growth_rate, dropout_rate=0.3):
        super(DenseBlock, self).__init__()
        
        # 1. 第一個 Conv: BN -> ReLU -> Conv
        self.conv1 = nn.Sequential(
            nn.BatchNorm2d(in_channels), # 期望輸入通道數
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False),
            nn.Dropout(dropout_rate) 
        )
        
        # 2. 第二個 Conv: BN -> ReLU -> Conv
        self.conv2 = nn.Sequential(
            nn.BatchNorm2d(growth_rate), # 輸入來自上一個 conv1 的輸出
            nn.ReLU(inplace=True),
            nn.Conv2d(growth_rate, growth_rate, kernel_size=3, padding=1, bias=False),
        )

    def forward(self, x):
        x1 = self.conv1(x) 
        x2 = self.conv2(x1)
        
        # 輸出是 (輸入 x) 和 (最終卷積輸出 x2) 的串聯
        return torch.cat([x, x2], dim=1) 


# --- 2. 堆疊單元: Residual Dense Block (RDB) ---
class ResDenseBlock(nn.Module):
    # RDB 的 in_channels 必須等於它的輸出通道數 (為滿足 U-Net 的跳躍連接要求)
    def __init__(self, in_channels, n_feature, n_layers=6, dropout_rate=0.3):
        super(ResDenseBlock, self).__init__()
        
        self.n_feature = n_feature 
        layers = []
        current_channels = in_channels # 從 RDB 的輸入通道開始
        
        for i in range(n_layers):
            # 每層 DB 的輸入是前一層的總輸出
            db = DenseBlock(current_channels, self.n_feature, dropout_rate)
            layers.append(db)
            # 通道數增加 n_feature
            current_channels += self.n_feature 

        self.dense_blocks = nn.Sequential(*layers)
        
        # 1x1 卷積過渡層 (Transition Layer): 
        # 將總輸出通道數 (current_channels) 轉換回 RDB 期望的輸出通道 (in_channels)，以進行殘差連接
        # 這是滿足 RDB 輸出通道 = 輸入通道的關鍵！
        self.transition_conv = nn.Conv2d(current_channels, in_channels, kernel_size=1, bias=False)

    def forward(self, x):
        dense_output = self.dense_blocks(x)
        processed_output = self.transition_conv(dense_output)
        
        # 殘差連接：輸出 = 壓縮後的密集輸出 + 原始輸入
        return processed_output + x


# --- 3. 整體架構：ResDense UNet (修正版 3) ---
class ResDenseUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(ResDenseUNet, self).__init__()
        
        # 論文參數 n: [8, 16, 32, 32, 16, 8]
        n_features = [8, 16, 32, 32, 16, 8] 
        
        # --- 通用層 ---
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        def create_upconv(in_c, out_c):
            # 使用 output_padding=1 確保正確的尺寸加倍 (例如 8->16, 16->32)
            return nn.ConvTranspose2d(in_c, out_c, kernel_size=3, stride=2, padding=1, output_padding=1)

        # --- Encoder (縮減路徑) ---
        
        # 1. 初始 Conv (3 -> 8)
        self.init_conv = nn.Conv2d(in_channels, n_features[0], kernel_size=1) 
        self.rdb1 = ResDenseBlock(in_channels=n_features[0], n_feature=n_features[0]) # Output C=8

        # 2. 通道調整 (Max Pool 後): 8 -> 16
        self.trans_conv1 = nn.Conv2d(n_features[0], n_features[1], kernel_size=1) 
        self.rdb2 = ResDenseBlock(in_channels=n_features[1], n_feature=n_features[1]) # Output C=16

        # 3. 通道調整 (Max Pool 後): 16 -> 32
        self.trans_conv2 = nn.Conv2d(n_features[1], n_features[2], kernel_size=1) 
        self.rdb3 = ResDenseBlock(in_channels=n_features[2], n_feature=n_features[2]) # Output C=32 (Bottleneck Input)

        # --- Decoder (擴增路徑) ---
        
        # 4. Stage 4: 8x8 -> 16x16
        # upconv4 輸入=Bottleneck 輸出(32), 輸出=n_features[3]=32 (用於 Concatenate)
        self.upconv4 = create_upconv(n_features[2], n_features[3]) 
        # rdb4 輸入: upconv(32) + skip(32) = 64
        self.rdb4 = ResDenseBlock(in_channels=n_features[3] + n_features[2], n_feature=n_features[3]) # Output C=64
        
        # 5. Stage 5: 16x16 -> 32x32
        # upconv5 輸入=rdb4_out(64), 輸出=n_features[4]=16
        self.upconv5 = create_upconv(n_features[3] + n_features[2], n_features[4])
        # rdb5 輸入: upconv(16) + skip(16) = 32
        self.rdb5 = ResDenseBlock(in_channels=n_features[4] + n_features[1], n_feature=n_features[4]) # Output C=32

        # 6. Stage 6: 32x32 -> 64x64
        # upconv6 輸入=rdb5_out(32), 輸出=n_features[5]=8
        self.upconv6 = create_upconv(n_features[4] + n_features[1], n_features[5])
        # rdb6 輸入: upconv(8) + skip(8) = 16
        self.rdb6 = ResDenseBlock(in_channels=n_features[5] + n_features[0], n_feature=n_features[5]) # Output C=16

        # --- 最終輸出層 (16 -> 1) ---
        self.final_conv = nn.Conv2d(n_features[5] + n_features[0], out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        # --- 打印 Shape 以供檢查 ---
        # print(f"Input: {x.shape}")
        
        # 1. Initial Conv & RDB1
        x = self.init_conv(x) 
        rdb1_out = self.rdb1(x) # Skip 1 (C=8, 64x64)
        
        # 2. Stage 2
        p1 = self.pool(rdb1_out) 
        t1 = self.trans_conv1(p1) # C: 8 -> 16
        rdb2_out = self.rdb2(t1) # Skip 2 (C=16, 32x32)
        
        # 3. Stage 3
        p2 = self.pool(rdb2_out)
        t2 = self.trans_conv2(p2) # C: 16 -> 32
        rdb3_out = self.rdb3(t2) # Skip 3 (C=32, 16x16)
        
        # 4. Bottleneck
        p3 = self.pool(rdb3_out) # Bottleneck (C=32, 8x8)

        # --- Decoder ---
        
        # 5. Stage 4: 8x8 -> 16x16
        up4 = self.upconv4(p3) # C=32
        cat4 = torch.cat([up4, rdb3_out], dim=1) # C=64
        rdb4_out = self.rdb4(cat4) # C=64
        
        # 6. Stage 5: 16x16 -> 32x32
        up5 = self.upconv5(rdb4_out) # C=16
        cat5 = torch.cat([up5, rdb2_out], dim=1) # C=32
        rdb5_out = self.rdb5(cat5) # C=32

        # 7. Stage 6: 32x32 -> 64x64
        up6 = self.upconv6(rdb5_out) # C=8
        cat6 = torch.cat([up6, rdb1_out], dim=1) # C=16
        rdb6_out = self.rdb6(cat6) # C=16

        # 8. Output
        return self.sigmoid(self.final_conv(rdb6_out))

In [86]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch

# 遵守論文設置的參數
BATCH_SIZE = 64
NUM_EPOCHS = 15 # 總訓練代數
INITIAL_LR = 0.0001
FINE_TUNE_LR = 0.00001 # 最低學習率 (min_lr)，模擬 Fine-tuning 階段

# 假設 paper_dice_loss 和 dice_score 已經被定義
# 假設 model, train_loader, val_loader, optimizer, device 已經被初始化

def train_model_optimized_v3(model, train_loader, val_loader, optimizer, num_epochs, device, keep_train_path=None):
    """
    優化後的模型訓練函數，支持學習率調度、早停和模型權重加載繼續訓練。

    Args:
        model (nn.Module): 待訓練的模型。
        train_loader (DataLoader): 訓練資料載入器。
        val_loader (DataLoader): 驗證資料載入器。
        optimizer (optim.Optimizer): 優化器。
        num_epochs (int): 總訓練代數。
        device (torch.device): 訓練設備 (如 'cuda' 或 'cpu')。
        keep_train_path (str, optional): 繼續訓練時加載的模型權重路徑。預設為 None。
    """

    # --- 1. 檢查並加載模型權重 ---
    if keep_train_path:
        try:
            model.load_state_dict(torch.load(keep_train_path, map_location=device))
            optimizer = torch.optim.Adam(model.parameters(), lr=INITIAL_LR)
            print(f"✅ 成功加載模型權重，從以下路徑繼續訓練: {keep_train_path}")
        except FileNotFoundError:
            print(f"⚠️ 錯誤: 未找到模型權重檔案於路徑: {keep_train_path}。將從頭開始訓練。")
        except Exception as e:
            print(f"⚠️ 加載模型權重時發生錯誤: {e}。將從頭開始訓練。")
    
    # --- 2. 初始化學習率調度器 ---
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='max', 
        factor=0.1, 
        patience=5, 
        verbose=True, 
        min_lr=FINE_TUNE_LR 
    )
    
    # 獨立的早停計數器 (用於完全停止訓練)
    early_stop_counter = 0
    early_stop_patience = 10 # 增加一個更大的寬限期，用於完全停止
    
    # 注意：如果從 Checkpoint 繼續訓練，這裡的 best_dice 應該從 Checkpoint 中讀取，
    # 但為簡化，這裡初始化為 0.0，假設目標是找到更好的性能。
    best_dice = 0.0 
    
    print(f"訓練開始：Batch Size={BATCH_SIZE}, 初始/當前 LR={optimizer.param_groups[0]['lr']:.7f}")
    
    # --- 3. 訓練迴圈 ---
    for epoch in range(num_epochs):
        # ------------------ 訓練階段 ------------------
        model.train() 
        train_loss_sum = 0
        
        for inputs, masks in train_loader:
            inputs, masks = inputs.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            # 確保 paper_dice_loss 和 dice_score 已被定義
            loss = paper_dice_loss(outputs, masks) 
            
            loss.backward()
            optimizer.step()
            
            train_loss_sum += loss.item() * inputs.size(0)
            
        train_loss_avg = train_loss_sum / len(train_loader.dataset)
        
        # ------------------ 驗證階段 ------------------
        model.eval() 
        val_loss_sum = 0
        val_dice_sum = 0
        
        with torch.no_grad():
            for inputs, masks in val_loader:
                inputs, masks = inputs.to(device), masks.to(device)
                outputs = model(inputs)
                
                loss = paper_dice_loss(outputs, masks)
                val_loss_sum += loss.item() * inputs.size(0)
                
                # 計算 Dice 係數
                val_dice_sum += dice_score(outputs, masks) * inputs.size(0)
                
        val_loss_avg = val_loss_sum / len(val_loader.dataset)
        val_dice_avg = val_dice_sum / len(val_loader.dataset)
        
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}/{num_epochs} (LR: {current_lr:.7f}): Train Loss={train_loss_avg:.4f}, "
              f"Val Loss={val_loss_avg:.4f}, Val DICE Score={val_dice_avg:.4f}")
        
        # ------------------ 學習率調度 ------------------
        scheduler.step(val_dice_avg)

        # ------------------ 模型保存與早停邏輯 ------------------
        
        if val_dice_avg > best_dice:
            best_dice = val_dice_avg
            early_stop_counter = 0 # 性能提升，重置早停計數器
            # 💡 最佳模型權重保存 (Checkpointing)
            torch.save(model.state_dict(), 'pth/best_stage1_unet_temp.pth')
            print("🚀 Dice Score 提升！保存最佳模型並重置早停計數。")
        else:
            # 性能未提升
            early_stop_counter += 1
            print(f"性能未提升。早停已停滯 {early_stop_counter} epochs。")
            
            # 檢查是否觸發完全停止
            if early_stop_counter >= early_stop_patience:
                # 檢查當前 LR 是否已經是最低 Fine-tuning LR
                if current_lr <= FINE_TUNE_LR * 1.001: 
                    print(f"🚨 早停觸發！驗證性能連續 {early_stop_patience} epochs 未提升，且已達最低 LR ({FINE_TUNE_LR})。訓練完全停止。")
                    return
                else:
                    print(f"⚠️ 早停計數 ({early_stop_counter}) 超過早期寬限期，但 LR 應在 {scheduler.patience} epochs 後調整。")


    print("總訓練代數已完成。")

In [88]:

# 遵守論文設置的參數
BATCH_SIZE = 64
NUM_EPOCHS = 15 # 總訓練代數
INITIAL_LR = 0.0001
FINE_TUNE_LR = 0.00001 # 最低學習率 (min_lr)，模擬 Fine-tuning 階段

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 模型與優化器設定 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResDenseUNet(in_channels=3, out_channels=1).to(device)

# 使用論文設置的初始學習率
optimizer = torch.optim.Adam(model.parameters(), lr=INITIAL_LR)

# --- 啟動訓練 ---
train_model_optimized_v3(model, train_loader, val_loader, optimizer, NUM_EPOCHS, device,None)

訓練開始：Batch Size=64, 初始/當前 LR=0.0001000
Epoch 1/15 (LR: 0.0001000): Train Loss=0.7461, Val Loss=0.6517, Val DICE Score=0.4347
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 2/15 (LR: 0.0001000): Train Loss=0.5374, Val Loss=0.5604, Val DICE Score=0.4714
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 3/15 (LR: 0.0001000): Train Loss=0.3423, Val Loss=0.5546, Val DICE Score=0.4510
性能未提升。早停已停滯 1 epochs。
Epoch 4/15 (LR: 0.0001000): Train Loss=0.2751, Val Loss=0.4545, Val DICE Score=0.5470
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 5/15 (LR: 0.0001000): Train Loss=0.2525, Val Loss=0.3875, Val DICE Score=0.6143
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 6/15 (LR: 0.0001000): Train Loss=0.2772, Val Loss=0.2887, Val DICE Score=0.7135
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 7/15 (LR: 0.0001000): Train Loss=0.2533, Val Loss=0.2474, Val DICE Score=0.7548
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 8/15 (LR: 0.0001000): Train Loss=0.2395, Val Loss=0.3859, Val DICE Score=0.6153
性能未提升。早停已停滯 1 epochs。
Epoch 9/15 (LR: 0.0001000): Train Loss=0.24

In [90]:
BATCH_SIZE = 64
NUM_EPOCHS = 60 # 總訓練代數
INITIAL_LR = 0.00001
FINE_TUNE_LR = 0.00002 # 最低學習率 (min_lr)，模擬 Fine-tuning 階段

path = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\pth\best_stage1_unet_temp0.pth"
train_model_optimized_v3(model, train_loader, val_loader, optimizer, NUM_EPOCHS, device, path)

C:\Users\ygz08\AppData\Local\Temp\ipykernel_3428\2979579552.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(keep_train_path, map_locati

✅ 成功加載模型權重，從以下路徑繼續訓練: D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\pth\best_stage1_unet_temp0.pth
訓練開始：Batch Size=64, 初始/當前 LR=0.0000100
Epoch 1/60 (LR: 0.0000100): Train Loss=0.2439, Val Loss=0.3874, Val DICE Score=0.6137
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 2/60 (LR: 0.0000100): Train Loss=0.2448, Val Loss=0.3863, Val DICE Score=0.6148
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 3/60 (LR: 0.0000100): Train Loss=0.2445, Val Loss=0.3537, Val DICE Score=0.6470
🚀 Dice Score 提升！保存最佳模型並重置早停計數。
Epoch 4/60 (LR: 0.0000100): Train Loss=0.2399, Val Loss=0.3630, Val DICE Score=0.6378
性能未提升。早停已停滯 1 epochs。
Epoch 5/60 (LR: 0.0000100): Train Loss=0.2339, Val Loss=0.4044, Val DICE Score=0.5963
性能未提升。早停已停滯 2 epochs。
Epoch 6/60 (LR: 0.0000100): Train Loss=0.2270, Val Loss=0.4061, Val DICE Score=0.5948
性能未提升。早停已停滯 3 epochs。
Epoch 7/60 (LR: 0.0000100): Train Loss=0.2306, Val Loss=0.3902, Val DICE Score=0.6104
性能未提升。早停已停滯 4 epochs。
Epoch 8/60 (LR: 0.0000100): Train Loss=0.2373, Val Loss=0.3526, Val DICE Sc